In [1]:
from google.colab import drive
drive.mount('/EMG_2Chs')

Mounted at /EMG_2Chs


In [2]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [3]:
# Diretório onde estão as pastas de classificação
classification_directory = '/EMG_2Chs/MyDrive/Colab Notebooks/EMG_2Chs/EMG-S1' # Coloque o caminho correto

# Lista para armazenar DataFrames de cada arquivo CSV
dataframes = []

# Loop pelas pastas de classificação
for classification_folder in os.listdir(classification_directory):
    classification_path = os.path.join(classification_directory, classification_folder)

    # Verifica se é uma pasta (ignora arquivos)
    if os.path.isdir(classification_path):
        # Loop pelos arquivos CSV na pasta de classificação
        for filename in os.listdir(classification_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(classification_path, filename)
                df = pd.read_csv(file_path)  # Lê o arquivo CSV como DataFrame

                # Adiciona o DataFrame à lista
                dataframes.append(df)

# Concatena todos os DataFrames em um único DataFrame "dataset"
dataset1 = pd.concat(dataframes, ignore_index=True)

In [4]:
dataset1

,EMG1,EMG2,class,user
0,0.000906,-3.858999e-07,9,1
1,0.000884,-9.461558e-06,9,1
2,0.000786,-2.485776e-05,9,1
3,0.000626,-4.300908e-05,9,1
4,0.000434,-5.791909e-05,9,1
...,...,...,...,...
1199995,0.000051,-6.154237e-04,0,1
1199996,0.000060,-5.719902e-04,0,1
1199997,0.000100,-5.384427e-04,0,1
1199998,0.000156,-5.147812e-04,0,1


In [ ]:
type(dataset1)

pandas.core.frame.DataFrame

In [5]:
import pandas as pd

# Supondo que você tenha um dataframe chamado 'dataset' contendo todos os dados
# e 'class' é a coluna que contém as classes

# Crie um dicionário para armazenar os dataframes separados por classe
dataframes_por_classe = {}

# Use a função groupby para agrupar os dados por classe
for classe, dados_classe in dataset1.groupby('class'):
    dataframes_por_classe[classe] = dados_classe.copy()

# Agora, 'dataframes_por_classe' é um dicionário onde as chaves são as classes e os valores são os dataframes correspondentes
# Cada dataframe mantém a coluna 'class'

# Se você quiser acessar um dataframe específico de uma classe (por exemplo, classe 0):
df_0 = dataframes_por_classe[0]
df_1 = dataframes_por_classe[1]
df_2 = dataframes_por_classe[2]
df_3 = dataframes_por_classe[3]
df_4 = dataframes_por_classe[4]
df_5 = dataframes_por_classe[5]
df_6 = dataframes_por_classe[6]
df_7 = dataframes_por_classe[7]
df_8 = dataframes_por_classe[8]
df_9 = dataframes_por_classe[9]




In [6]:
emg_data = dataset1

In [7]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [8]:
# Função para calcular Slope Sign Changes (SSC)
def calculate_ssc(signal):
    diff = np.diff(np.sign(np.diff(signal)))
    return np.sum(diff != 0)

# Função para calcular Number of Zero Crossings (ZC)
def calculate_zc(signal):
    return np.sum(np.diff(np.sign(signal)) != 0)

# Função para calcular Waveform Length (WL)
def calculate_wl(signal):
    return np.sum(np.abs(np.diff(signal)))

# Função para calcular Hjorth Time Domain Parameters (HTD)
def calculate_htd(signal):
    mean = np.mean(signal)
    diff = np.diff(signal)
    diff_mean = np.mean(diff)
    diff_diff = np.diff(diff)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff) / activity)
    complexity = np.sqrt(np.var(diff_diff) / np.var(diff) / mobility)

    return activity, mobility, complexity

# Função para calcular Sample Skewness (SS)
def calculate_ss(signal):
    return skew(signal)

# Função para calcular AutoRegressive Model Parameters (AR)
def calculate_ar(signal, order=4):
    # Use a função autoregressiva do numpy
    ar_coefficients = np.polyfit(signal[:-1], signal[1:], order)
    return ar_coefficients

In [9]:
def sliding_window(data, window_size, window_increment):
    features = []
    for i in range(0, len(data) - window_size + 1, window_increment):
        window = data[i:i + window_size]
        # Extrair características da janela (por exemplo, RMS, ZC, WL, HTD, SS, AR)
        extracted_features = extract_features_from_window(window)
        features.append(extracted_features)
    return np.array(features)

def extract_features_from_window(window):
    # Implemente as funções para calcular as características (por exemplo, SSC, ZC, WL, HTD, SS, AR)
    # Certifique-se de ajustar de acordo com as características específicas do seu artigo
    ssc_feature = calculate_ssc(window)
    zc_feature = calculate_zc(window)
    wl_feature = calculate_wl(window)
    htd_feature = calculate_htd(window)
    ss_feature = calculate_ss(window)
    ar_feature = calculate_ar(window)
    return [ssc_feature, zc_feature, wl_feature, htd_feature, ss_feature, ar_feature]


In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def concatenate_and_lda(emg_channel1, emg_channel2, window_size, window_increment):
    features_channel1 = sliding_window(emg_channel1, window_size, window_increment)
    features_channel2 = sliding_window(emg_channel2, window_size, window_increment)

    # Concatenar características de ambos os canais
    concatenated_features = np.concatenate((features_channel1, features_channel2), axis=1)

    # Aplicar LDA para redução de dimensionalidade
    lda = LinearDiscriminantAnalysis(n_components=9)
    reduced_features = lda.fit_transform(concatenated_features, labels)  # Certifique-se de ter rótulos adequados

    return reduced_features


In [11]:
# Defina o tamanho da janela e o incremento conforme necessário
window_size = 100
window_increment = 50

# Lista para armazenar os resultados para cada classe
all_reduced_features = []

# Iterar sobre todos os DataFrames correspondentes às diferentes classes
for i in range(10):  # Supondo que você tenha 10 classes numeradas de 0 a 9
    # Nome da classe e dataframe correspondente
    class_name = f'df_{i}'
    current_df = globals()[class_name]  # Obtenha o DataFrame pelo nome

    # Supondo que você tenha rótulos nas colunas 'class'
    labels = current_df['class'].values

    # Extrair características e aplicar LDA para a classe atual
    reduced_features = concatenate_and_lda(current_df['EMG1'], current_df['EMG2'], window_size, window_increment, labels)

    # Armazenar os resultados
    all_reduced_features.append(reduced_features)

# 'all_reduced_features' agora contém os resultados para todas as classes


TypeError: ignored